In [ ]:
"""
Analysis of Hydra polyp single cell RNAseq data using ScanPy
Marc Meynadier
"""

In [ ]:
!date

In [ ]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy.sparse
import scanpy.external as sce
import os

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import (KNeighborsClassifier,NeighborhoodComponentsAnalysis)
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib import rcParams
#import plotly.tools as tls

import seaborn as sns
sns.set(style="whitegrid")

import collections
import scipy.sparse as sp_sparse
import h5py

In [ ]:
sc.logging.print_header()
sc.settings.set_figure_params(dpi=150, facecolor='white')

In [ ]:
# Loading polyp data

homeostatic1Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440173/"
homeostatic2Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440174/"
homeostatic3Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440175/"
homeostatic4Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440176/"
homeostatic5Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440177/"
homeostaticFemaleSpikeInPath = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440178/"
homeostatic6Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440179/"
homeostaticMaleSpikeInPath = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440180/"
homeostaticFemalePath = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440181/"
homeostatic7Path = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440182/"
homeostaticMalePath = "../../../../species/Hydra/analysis/STARmapping/siebert2019/GSE121617/GSM3440183/"

homeostatic1= sc.read_10x_mtx(homeostatic1Path) ; homeostatic1.var_names_make_unique()
homeostatic2 = sc.read_10x_mtx(homeostatic2Path) ; homeostatic2.var_names_make_unique()
homeostatic3 = sc.read_10x_mtx(homeostatic3Path) ; homeostatic3.var_names_make_unique()
homeostatic4 = sc.read_10x_mtx(homeostatic4Path) ; homeostatic4.var_names_make_unique()
homeostatic5 = sc.read_10x_mtx(homeostatic5Path) ; homeostatic5.var_names_make_unique()
homeostaticFemaleSpikeIn = sc.read_10x_mtx(homeostaticFemaleSpikeInPath) ; homeostaticFemaleSpikeIn.var_names_make_unique()
homeostatic6 = sc.read_10x_mtx(homeostatic6Path) ; homeostatic6.var_names_make_unique()
homeostaticMaleSpikeIn = sc.read_10x_mtx(homeostaticMaleSpikeInPath) ; homeostaticMaleSpikeIn.var_names_make_unique()
homeostaticFemale = sc.read_10x_mtx(homeostaticFemalePath) ; homeostaticFemale.var_names_make_unique()
homeostatic7 = sc.read_10x_mtx(homeostatic7Path) ; homeostatic7.var_names_make_unique()
homeostaticMale = sc.read_10x_mtx(homeostaticMalePath) ; homeostaticMale.var_names_make_unique()

In [ ]:
# Metadata
homeostatic1.obs['type']="Polyp"
homeostatic1.obs['sample']="Homeostatic rep 1"

homeostatic2.obs['type']="Polyp"
homeostatic2.obs['sample']="Homeostatic rep 2"

homeostatic3.obs['type']="Polyp"
homeostatic3.obs['sample']="Homeostatic rep 3"

homeostatic4.obs['type']="Polyp"
homeostatic4.obs['sample']="Homeostatic rep 4"

homeostatic5.obs['type']="Polyp"
homeostatic5.obs['sample']="Homeostatic rep 5"

homeostaticFemaleSpikeIn.obs['type']="Polyp"
homeostaticFemaleSpikeIn.obs['sample']="Homeostatic female spike-in"

homeostatic6.obs['type']="Polyp"
homeostatic6.obs['sample']="Homeostatic rep 6"

homeostaticMaleSpikeIn.obs['type']="Polyp"
homeostaticMaleSpikeIn.obs['sample']="Homeostatic male spike-in"

homeostaticFemale.obs['type']="Polyp"
homeostaticFemale.obs['sample']="Homeostatic female"

homeostatic7.obs['type']="Polyp"
homeostatic7.obs['sample']="Homeostatic rep 7"

homeostaticMale.obs['type']="Polyp"
homeostaticMale.obs['sample']="Homeostatic male"

In [ ]:
# Merging
adata = homeostatic1.concatenate(homeostatic2,homeostatic3,homeostatic4,homeostatic5,homeostaticFemaleSpikeIn,homeostatic6,homeostaticMaleSpikeIn,homeostaticFemale,homeostatic7,homeostaticMale,join='outer') 
print(adata.obs['sample'].value_counts())

In [ ]:
# Mitochondrial genes
mitoGenes = pd.read_csv('../../../../species/Hydra/raw/HydraMitoGenes.txt',sep='\t')
mitoGenes = list(set(list(mitoGenes.iloc[:,0])))
print(mitoGenes)

adata.var['mt'] = False
adata.var = adata.var.reset_index()
for idx, row in adata.var.iterrows():
    for j in mitoGenes:
        if row['index'] == j:
            adata.var.at[idx,'mt'] = True
adata.var = adata.var.set_index('index')

print(adata.var)
adata.var.mt.value_counts()

In [ ]:
# Mitochondrial quality check
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)
mito_genes = adata.var['mt'].to_numpy()
# For each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mt2'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# Add the total counts per cell as observations-annotation 
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
adata

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts'],
             jitter=0.4, groupby = 'sample', rotation= 45, ncols=2)
sc.pl.violin(adata, ['pct_counts_mt', 'percent_mt2','log1p_total_counts_mt'],
             jitter=0.4, groupby = 'sample', rotation= 45, ncols=2)

In [ ]:
# Plot quality check measures
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color="sample")
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color="sample")
sc.pl.scatter(adata, x='log1p_total_counts', y='log1p_n_genes_by_counts', color="sample")

In [ ]:
# Filtering by genes counts and mitochrondial genes counts
adata = adata[adata.obs.n_genes_by_counts < 9000, :]
adata = adata[adata.obs.pct_counts_mt < 1, :]
adata

In [ ]:
# Filtering by number of counts and number of cells
sc.pp.filter_cells(adata, min_counts=500)
sc.pp.filter_genes(adata, min_cells=2)
print(adata.n_obs,adata.n_vars)

In [ ]:
# Plotting percentage of counts per gene 
sc.pl.highest_expr_genes(adata, n_top=50)

In [ ]:
# Exporting raw adata
rPath = os.getcwd()
outputPath = rPath+'/../../../../species/Hydra/analysis/ScanPy/'
adata.write_h5ad(outputPath+'Hydra_polyp_raw.h5ad')

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata.raw = adata

adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Correcting batch effet
sce.pp.harmony_integrate(adata,'sample',max_iter_harmony=20)

In [ ]:
adata

In [ ]:
#plot the amount of variance explained by each PC
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50)

In [ ]:
# Computing neighbors graph
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=27, knn=True, use_rep='X_pca_harmony')

# Clustering the graph using Leiden algorithm
sc.tl.leiden(adata,resolution=1.2) 

# Computing UMAP
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['leiden','sample'], title="UMAP Hydra polyp",ncols=1)
sc.pl.umap(adata, color=['leiden'], legend_loc= 'on data')
sc.pl.umap(adata, color=['total_counts', 'n_genes_by_counts'])
sc.pl.umap(adata, color=['log1p_total_counts', 'log1p_n_genes_by_counts'])

In [ ]:
# Checking how each sample contribute to the final map => investigate possible batch effect

sc.pl.umap(adata, color=['leiden'], frameon=False)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 1'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 2'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 3'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 4'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 5'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic female spike-in'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 6'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic male spike-in'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic female'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic rep 7'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)
sc.pl.umap(adata[adata.obs.loc[adata.obs['sample'].isin(['Homeostatic male'])].index.tolist(), ], color= ['leiden','sample'], title=['Clusters','sample'], frameon=False, size=10,ncols=1)

In [ ]:
# Re-exporting data
adata.write_h5ad(outputPath+'Hydra_polyp.h5ad')

In [ ]:
# Ranking and extracting marker genes for all clusters
sc.tl.rank_genes_groups(adata,'leiden',n_genes=1000,method='wilcoxon')

clusters = np.unique(adata.obs['leiden'])
clusters = clusters.astype('int')
clusters = np.sort(clusters)
clusters = clusters.astype('str')

markers = pd.DataFrame()
clus = []
markerGene = []
padj = []
for i in clusters:
  genes = adata.uns['rank_genes_groups']['names'][str(i)]
  clus += list(np.repeat(i,len(genes)))
  markerGene += list(genes)
  padj += list(adata.uns['rank_genes_groups']['pvals_adj'][str(i)])
markers['clus'] = clus
markers['markerGene'] = markerGene
markers['padj'] = padj
significant = []
for index,row in markers.iterrows():
    if row['padj'] < 0.05:
        significant += "T"
    else:
        significant += "F"
markers['significant'] = significant
significant_markers = markers[markers.significant == "T"]
significant_markers.to_csv(outputPath+'Hydra_polyp_markers_1000g.csv', sep='\t',index=False)
print(significant_markers)

In [ ]:
def cluster_markers(markersDf,clusterNumber):
    subset = markersDf[markersDf["clus"].isin(clusterNumber)]
    markerGene = subset["markerGene"].tolist()
    return markerGene

def save_markers(markerList,path,clusterName):
    with open(path+"Hydra_polyp_"+clusterName+"_markers.txt","w") as f:
        for i in markerList:
            f.write(i+"\n")
        f.close

In [ ]:
# Plotting top marker gene for each cluster
markers = []
for i in np.unique(adata.obs['leiden']):
     markers += [adata.uns['rank_genes_groups']['names'][str(i)][0]]
sc.pl.umap(adata,color=markers,color_map = 'viridis')

In [ ]:
sc.tl.paga(adata, groups='leiden')

In [ ]:
adata.obs['leiden_anno'] = adata.obs['leiden']

In [ ]:
adata.obs['leiden_anno'].cat.categories = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36']

In [ ]:
sc.pl.paga(adata, threshold=0.03, show=False,fontsize=9)

In [ ]:
# Test
sc.pl.umap(adata,color=['Sc4wPfr_388.g4803'],legend_loc= 'on data')

In [ ]:
cluster_markers(significant_markers,['5'])

In [ ]:
# Mucous gland
#sc.pl.umap(adata,color=['Xe_021837'],legend_loc= 'on data') # C8 (mucin)

In [ ]:
# Epidermis
#sc.pl.umap(adata,color=['Xe_019742'],legend_loc= 'on data')

In [ ]:
# Neurons
sc.pl.umap(adata,color=['Sc4wPfr_301.g13998','Sc4wPfr_149.g26107','Sc4wPfr_162.g21720','Sc4wPfr_332.g19470','Sc4wPfr_251.g2490','Sc4wPfr_246.g4301','Sc4wPfr_559.g507'],legend_loc= 'on data') 

In [ ]:
# Cnidocytes
sc.pl.umap(adata,color=['Sc4wPfr_390.g5618','Sc4wPfr_761.g1924','Sc4wPfr_239.g21141','Sc4wPfr_1007.g33053','Sc4wPfr_786.g1708','Sc4wPfr_706.g27778'],legend_loc= 'on data') 

In [ ]:
# Gastroderm 
#sc.pl.umap(adata,color=['Xe_005638','Xe_005640'],legend_loc= 'on data')

In [ ]:
# Digestive gland cells
#sc.pl.umap(adata,color=['Xe_013237'],legend_loc= 'on data')

In [ ]:
# Mature cnidocyte
#sc.pl.umap(adata,color=['Xe_002963'],legend_loc= 'on data') # By blast on Nematostella + pfam => Anoctamin

In [ ]:
# Muscle
#sc.pl.umap(adata,color=['Xe_014332','Xe_011435','Xe_013837'],legend_loc= 'on data') # Calponin, tropomyosin

In [ ]:
# Endosymbiotic cells
#sc.pl.umap(adata,color=['Xe_000857','Xe_015355','Xe_028058'],legend_loc= 'on data') #Granulin, CD36, Tachylectin

In [ ]:
#adata.obs['leiden_anno'].cat.categories = ["0 - Gastrodermis","1 - Ectodermis","2 - Ectodermis","3 - Epidermis","4 - Ectodermis","5 - Ectodermis","6 - NPC","7 - Muscle","8 - Ectodermis","9 - Ectodermis","10 - Gastrodermis","11 - Neurons","12 - Cnidocytes","13 - Epidermis","14 - Mature cnidocytes","15 - Secretory cells (neurosecretory ?)","16 - Ectodermis (secretory ?)","17 - Neurosecretory cells","18 - Early cnidocytes","19 - Mucous gland","20 - Endosymbiotic cells","21 - ?","22 - Endosymbiotic cells","23 - Neurons"]

In [ ]:
#sc.pl.umap(adata, color=['leiden_anno'],title='Hydra polyp annotated clusters',ncols=1,frameon=False)

In [ ]:
#sc.pl.umap(adata,title='Hydra polyp', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#A366E0", "13":"#E2E2E2", "14":"#A366E0", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#A366E0", "19":"#E2E2E2", "20":"#E2E2E2", "21":"#E2E2E2", "22":"#E2E2E2", "23":"#E2E2E2"}, frameon=False,save="cnidocytesHydraPolyp")

In [ ]:
#sc.pl.umap(adata,title='Hydra polyp', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#0080ff", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2", "19":"#E2E2E2", "20":"#E2E2E2", "21":"#E2E2E2", "22":"#E2E2E2", "23":"#0080ff"}, frameon=False,save="neuronsHydraPolyp")

In [ ]:
sc.settings.set_figure_params(dpi=150, facecolor='white')
sc.pl.umap(adata,title='Hydra polyp\nRBH - one2one - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#A366E0", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#A366E0", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#A366E0", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2","21":"#A366E0","22":"#E2E2E2","23":"#E2E2E2","24":"#A366E0","25":"#E2E2E2","26":"#A366E0","27":"#A366E0","28":"#E2E2E2","29":"#E2E2E2","30":"#E2E2E2","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#E2E2E2","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nRBH - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#A366E0", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#A366E0", "16":"#E2E2E2", "17":"#A366E0", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2","21":"#A366E0","22":"#E2E2E2","23":"#E2E2E2","24":"#A366E0","25":"#E2E2E2","26":"#A366E0","27":"#A366E0","28":"#E2E2E2","29":"#E2E2E2","30":"#E2E2E2","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#E2E2E2","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nOrthoFinder - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#A366E0", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#A366E0", "16":"#E2E2E2", "17":"#A366E0", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2","21":"#A366E0","22":"#E2E2E2","23":"#E2E2E2","24":"#A366E0","25":"#E2E2E2","26":"#A366E0","27":"#E2E2E2","28":"#E2E2E2","29":"#E2E2E2","30":"#E2E2E2","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#E2E2E2","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nOrthoFinder - many2many - genomeBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#A366E0", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#A366E0", "16":"#E2E2E2", "17":"#A366E0", "18":"#E2E2E2","19":"#E2E2E2","20":"#E2E2E2","21":"#A366E0","22":"#E2E2E2","23":"#E2E2E2","24":"#A366E0","25":"#E2E2E2","26":"#A366E0","27":"#E2E2E2","28":"#E2E2E2","29":"#E2E2E2","30":"#E2E2E2","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#E2E2E2","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nRBH - one2one - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#0080ff", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#0080ff","21":"#E2E2E2","22":"#E2E2E2","23":"#E2E2E2","24":"#E2E2E2","25":"#0080ff","26":"#E2E2E2","27":"#E2E2E2","28":"#E2E2E2","29":"#0080ff","30":"#E2E2E2","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#E2E2E2","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nRBH - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#0080ff", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#0080ff","21":"#E2E2E2","22":"#E2E2E2","23":"#E2E2E2","24":"#E2E2E2","25":"#0080ff","26":"#E2E2E2","27":"#E2E2E2","28":"#E2E2E2","29":"#0080ff","30":"#E2E2E2","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#E2E2E2","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nOrthoFinder - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#0080ff", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#0080ff","21":"#E2E2E2","22":"#E2E2E2","23":"#E2E2E2","24":"#E2E2E2","25":"#0080ff","26":"#E2E2E2","27":"#E2E2E2","28":"#E2E2E2","29":"#0080ff","30":"#0080ff","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#0080ff","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Hydra polyp\nOrthoFinder - many2many - genomeBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#0080ff", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2","19":"#E2E2E2","20":"#0080ff","21":"#E2E2E2","22":"#E2E2E2","23":"#E2E2E2","24":"#E2E2E2","25":"#0080ff","26":"#E2E2E2","27":"#E2E2E2","28":"#E2E2E2","29":"#0080ff","30":"#0080ff","31":"#E2E2E2","32":"#E2E2E2","33":"#E2E2E2","34":"#0080ff","35":"#E2E2E2","36":"#E2E2E2"}, frameon=False)